In [7]:
from tensorflow.keras.activations import elu,exponential,gelu,linear,relu,selu,sigmoid,softmax,softplus,swish,tanh
from tensorflow.math import add,atan,cos,erf,maximum,minimum,sin,sqrt,subtract
from deap.gp import PrimitiveSet,PrimitiveTree,genGrow,genFull,compile,cxOnePoint,mutShrink,staticLimit
from deap.algorithms import eaSimple
from deap import creator,base,tools 
from copy import deepcopy
from collections import Counter
import numpy as np 
import tensorflow as tf 
import operator 
np.set_printoptions(precision=2)

# We define the Evolutionary Algorithm class, which encapsulates everything that has to do with the evolution. This class has access to a LangevinEBM object, which implements a function that trains an EBM using Langevin Dynamics and the other bells and whistles found in the OpenAI paper, and then returns a scalar score that is directly proportional to the fitness of an individual. 

In [8]:
class EvolutionaryAlgorithm:
    
    def __init__(self,base_act_functions,base_operations,min_depth,max_depth,pop_size):
        
        # We first initialize the pset that contains our base 
        # building blocks.
        
        self.base_act_functions=base_act_functions
        self.base_operations=base_operations
        self.pset=self.initialize_pset(base_act_functions,base_operations)
        self.pop_size=pop_size 
        
        # We specify that we are dealing with a maximization problem, and 
        # that our individual is a PrimitiveTree. We add a reference 
        # to the pset in "Individual", since it is used by some DEAP 
        # GP operators when modifying an individual.
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual",PrimitiveTree,fitness=creator.FitnessMax,pset=self.pset)
        
        self.toolbox=base.Toolbox()
        
        # Our toolbox contains an "expr" function that calls genGrow with 
        # the pset as a parameter. genGrow also takes as parameter the 
        # minimum and maximum depth of the tree. We do not fix them
        # here, and we leave them as variables which can be changed when 
        # creating a population with the create_generation() function
        
        self.toolbox.register("expr",genGrow,pset=self.pset,min_=min_depth,max_=max_depth)
        
        #The "individual" function calls "expr()" on a "creator.Individual" object 
        # and returns it. In other words, it simply creates a new Primitive tree 
        # from our pset
        self.toolbox.register("individual",tools.initIterate,creator.Individual,self.toolbox.expr)
        
        # We register the variation operators and a simple EA algorithm.
        
        self.toolbox.register("evaluate",self.get_ebm_fitness)
        self.toolbox.register("select",tools.selRoulette)
        self.toolbox.register("mate",cxOnePoint)
        self.toolbox.register("mutate",mutShrink)
        
        # We decorate the mating and mutation operator with a limit on the maximum depth.
        
        self.toolbox.decorate("mate",staticLimit(key=operator.attrgetter("height"),max_value=5))
        self.toolbox.decorate("mutate",staticLimit(key=operator.attrgetter("height"),max_value=5))
        
        stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
        stats_size = tools.Statistics(len)
        self.mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
        self.mstats.register("avg", np.mean)
        self.mstats.register("std", np.std)
        self.mstats.register("min", np.min)
        self.mstats.register("max", np.max)
        
        # We also add a Hall of fame object, and we keep the 10 best individuals in it.
        
        self.hof=tools.HallOfFame(10)
        
        
        
    def initialize_pset(self,base_act_functions,base_operations):
        
        # Our desired functions are unary and we specificy this when creating the pset
        pset=PrimitiveSet("main",1)
        
        # Activation functions in a neural network have 
        # arity 1.
        for func in base_act_functions:
            pset.addPrimitive(func,1)
            
        # The operations that we are considering, such as 
        # maximum, add or subtract, have arity 2.
        for op in base_operations:
            pset.addPrimitive(op,2)
            
        pset.renameArguments(ARG0="x")
        
        return pset

    def adjust_activation_tree(self,activation_tree):
        copied_activation_tree = deepcopy(activation_tree)
        bias_pset = PrimitiveSet(name='bias_set', arity=1)
        for func in self.base_act_functions:
            bias_pset.addTerminal(terminal=func, name='terminal_' + func.__name__)
        for func in self.base_act_functions:
            bias_pset.addPrimitive(primitive=func, arity=1, name=func.__name__)
        primitives_dictionary = {}
        for bias_primitive in bias_pset.primitives[object]:
            for primitive in self.pset.primitives[object]:
                if bias_primitive.name == primitive.name:
                    primitives_dictionary[bias_primitive.name] = primitive
        tree_elements = np.vectorize(lambda x: x.name)
        num_terminals = Counter(tree_elements(copied_activation_tree))['ARG0']
        starting_index = 0
        two_literals = False
        for i in range(num_terminals):
            done = False
            while not done:
                bias_activation = PrimitiveTree(genFull(pset=bias_pset, min_=1, max_=1))
                if 'ARG0' not in tree_elements(bias_activation):
                    done = True
            for i in range(starting_index, len(copied_activation_tree)):
                if i < len(copied_activation_tree) - 1:
                    condition = copied_activation_tree[i].name == 'ARG0' and \
                                (copied_activation_tree[i + 1].name == 'ARG0' or two_literals)
                else:
                    condition = copied_activation_tree[i].name == 'ARG0' and two_literals
                if condition:
                    if i < len(copied_activation_tree) - 1:
                        two_literals = copied_activation_tree[i + 1].name == 'ARG0'
                    replace_by_activation = np.random.choice([True, False], p=[0.8, 0.2])
                    if replace_by_activation:
                        starting_index = i + 2
                        activation_primitive = primitives_dictionary[bias_activation[0].name]
                        copied_activation_tree.insert(i, activation_primitive)
                    else:
                        starting_index = i + 1
                    break
        return copied_activation_tree
    
    def create_generation(self):
        generation=[self.toolbox.individual() for i in range(self.pop_size)]
        generation=[self.adjust_activation_tree(x) for x in generation]
        return generation
    
    # This will be replaced with a proper EBM training function later on. Make sure to take care of nan case 
    
    def get_ebm_fitness(self,individual):
        
        # Choose some very bad value in case of an error, not infinity to prevent DEAP from encountering an error
        f=compile(individual,self.pset)
        const=tf.constant([1.0,2.0,3.0])
        fitness=np.around(tf.reduce_sum(f(const)).numpy(),2)
        if np.isnan(fitness) or np.isinf(fitness):
            fitness= -10**3
        return (fitness,)
    
    def evolve_functions(self,num_generations):
        pop=self.create_generation()
        pop,log=eaSimple(pop,self.toolbox,ngen=num_generations,cxpb=0.8,mutpb=0.02,stats=self.mstats,halloffame=self.hof,verbose=True)
        return pop
    
    def get_hof_inds(self):
        return ['Function:{}\tFitness:{}'.format(str(mvp),mvp.fitness.values[0]) for mvp in self.hof]

In [9]:
base_functions=[elu,gelu,linear,relu,selu,sigmoid,softplus,swish,tanh,atan,cos,erf,sin,sqrt]
base_operations=[maximum,minimum,add,subtract]

In [10]:
evo=EvolutionaryAlgorithm(base_functions,base_operations,min_depth=1,max_depth=3,pop_size=10)

/home/andy/.local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/andy/.local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [11]:
#individuals=evo.create_generation()
#for ind in individuals:
#    print(str(ind))
#for ind in individuals:
#    print(evo.get_ebm_fitness(ind))

In [12]:
pop=evo.evolve_functions(10)
print("\n\n")

hof_info=evo.get_hof_inds()
for mvp_info in hof_info:
    print(mvp_info)

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg  	gen	max 	min 	nevals	std    	avg	gen	max	min	nevals	std    
0  	10    	4.427	0  	9.61	2.16	10    	2.18193	3.4	0  	7  	2  	10    	1.68523
1  	6     	5.054	1  	9.61	2.56	6     	2.11892	3.2	1  	7  	2  	6     	1.72047
2  	8     	5.246	2  	9.61	2.14	8     	2.19194	3.2	2  	7  	2  	8     	1.72047
3  	8     	6.691	3  	12.61	3.14	8     	2.8229 	3.9	3  	8  	2  	8     	1.92094
4  	6     	8.476	4  	12.61	3.14	6     	2.83517	4.5	4  	6  	2  	6     	1.28452
5  	8     	8.636	5  	12.61	6   	8     	2.71782	4.5	5  	6  	2  	8     	1.68819
6  	6     	8.859	6  	12.61	3.14	6     	2.85744	4.9	6  	6  	2  	6     	1.3    
7  	6     	10.491	7  	12.61	6   	6     	2.46412	5.5	7  	8  	2  	6     	1.56525
8  	8     	11.602	8  	12.61	9.14	8     	1.13205	5.2	8  	8  	4  	8     	1.4    
9  	8   

# Abhishek: What I expect from you:

# 1) Define an EBM class that has a get_ebm_fitness(activation_function,train_test,val_test) function that returns a scalar directly proportional to the goodness.
# 2) Potential example of such a class
# 3) Be very careful about compatibility issues. 

In [ ]:
class EBMProbML:
    
    def train_ebm(self,mode,activation_function=tf.keras.activations.relu,train_set,val_set):
        # It returns a scalar value that is DIRECTLY PROPORTIONAL to the "goodness"
        # Train on train set, evaluate on validation set.
        # All of these modes and their measure of fitness are described in the main paper
        if mode=='unconditional':
            return train_cifar10_unconditional(...)
        elif mode=='condtional':
            return train_cifar10_conditional(...)
        elif mode=='train_ood':
            return train_ood(...)
        
    def train_cifar10_unconditional(self,...):
        return fitness
    
    def train_cifar10_conditional(self,...):
        return fitness
    
    def train_ood(self,...):
        return fitness